In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore as PC
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

Extracting Data from the PDF

In [2]:
def load_pdf(data):
    loader=DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf("C:\VS Workspace\Medical ChatBOT\data")

Creating Text Chunks

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("length of my chunk is: ", len(text_chunks))

Downloading Embedding-Model from HuggingFace

In [9]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings=download_hugging_face_embeddings()

Initializing the Pinecone

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')

In [12]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

Creating Pinecone Index

In [27]:
index_name = "medical-chatbot" 
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
) 

In [28]:
index = pc.Index(index_name)

Creating Vector Embedding of the Text Chunks and storing in the Pinecone Database

In [29]:
docsearch=PC.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

Answering the Query with the help of Simiarity-Search

In [ ]:
docsearch=PC.from_existing_index(index_name, embeddings)
query="What are allergies"
docs=docsearch.similarity_search(query, k=3)
print(docs)

In [31]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [32]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [33]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [34]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])